---
title: Using Machine-Learning tools to Classify aDNA Damage
author: Emrah Kırdök, Ph.D.

---

## Introduction

One of our aims in this project is to distinguis aDNA reads from the mDNA using machine-learing tools. 

Here I am documenting Marie Gasproux's idea to train clasifier using *Vowpal Wobbit*. Her idea is to train a classifier to classify *Viral* sequences. I would like to adapt this code to train a Vowpal wobbit classifier to distinguis aDNA reads from modern reads.

As a test, I used Mycobacterium aDNA and mDNA for this purpose in *cbib* servers.

## Code

The definitions:

In [42]:
from collections import defaultdict

Here is the function for the spaced *k-mer* idea. For a non-spaced *k-mer* we should use 111. For a spaced *k-mer* we should use 101. So it basically transforms 0 to \*. I should work on this idea.

In [43]:
def space_kmer(kmer, pattern):
    return "".join(letter if bit=='1'
                   else '*'
                   for letter, bit in zip(kmer, pattern))

This is the function to count *k-mers* that Marie wrote. It is a straight-forward approach and similiar that I used before.

She used a `ref_read_size`variable to normalize *k-mer* counts. I should understand this normalization or just omit it. Since I have a standart size of 150 in my reads, this parameter will not affect the studies.

This function returns a dictionary in this format:

```
*k-mer*: *count*
```

In [44]:
def kmerize(read, pattern, ref_read_size):
    d = defaultdict(float)
    lp = len(pattern)
    max_pos = len(read)-lp+1
    for i in range(max_pos):
        d[space_kmer(read[i:i+lp], pattern)] += 1
    coef = len(read)/ref_read_size
    return {k:v*coef for k, v in d.items()}

Lets start to use the functions. One important thing is, pattern should be a `str` to calculate the length.

In [45]:
seq="CGACTGTCGTCAGTCAGTCAGTCAGTCAGTCAGTCAGTCATGACGTAGTCAGT"
length_seq=len(seq)
pattern=str(111)

Lets count the *k-mers*:

In [46]:
features= kmerize(seq,pattern,length_seq)

To feed this information to *Vowpal Wobbit*, we need to print the counts like that:

category 'label | feature-1:information feature-2:information ...

category: Is an integer and the algorithm uses this variable to label the information. If left empty, it means that you want to assign a label.

'label: This is the human readable information

This function takes a dictionary and transforms it to the desired format. But the 

In this function:

g_label: is the human readable formay
sk_dict: the dictionary
g_cat: is the category label that algorithm uses to classify information

In [47]:
def print_vw_features(g_label, sk_dict, g_cat=None):
    if g_cat != None:
        feats = "%d '%d |" % (g_cat, g_label)
    else:
        feats = "'%d |" % g_label
    for k, v in sorted(sk_dict.items()):
        feats += " %s:%.2f" % (k, v)
    return feats + "\n"

This is how we use this code

In [48]:
print_vw_features(1,features,1)

"1 '1 | ACG:1.00 ACT:1.00 AGT:9.00 ATG:1.00 CAG:8.00 CAT:1.00 CGA:1.00 CGT:2.00 CTG:1.00 GAC:2.00 GTA:1.00 GTC:10.00 TAG:1.00 TCA:9.00 TCG:1.00 TGA:1.00 TGT:1.00\n"

Next steps are just making a program to pratically process all the sequnces. I derived the code from my kount.py script.